In [1]:
import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
from importlib import reload
# import [module]
# importlib.reload([module])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
def show(x):
    plt.imshow(x)
    plt.show()

In [4]:
import custom.models
reload(custom.models)
from custom.models import u_net, u_net_res, seg_net

In [5]:
from custom.models import u_net, seg_net
from custom.utils import conditioned_inverse
from custom.datasets import nuclei_dataset, nuclei_dataset_gen
from custom.loss import weighted_bce_loss_with_logits as loss_func

/root/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
import time
ts = time.time()
nuclei_dataset_df = nuclei_dataset_gen(path='../data/stage1_train/')
te = time.time()
print(te - ts)

645.9010012149811


In [7]:
dataset = nuclei_dataset(nuclei_dataset_df)

In [8]:
train_loader = DataLoader(dataset,batch_size=16, shuffle=True)
test_loader = DataLoader(nuclei_dataset(nuclei_dataset_df, train=False), batch_size=16)

In [10]:
# del model
torch.cuda.empty_cache()

In [11]:
# model = u_net(bn=True)
# model = u_net_res()
# model = u_net(bn=False)
# model.load_state_dict(torch.load('./models/unbn_1'))
model = seg_net()
model.cuda()

seg_net(
  (conv1_3): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_5): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_7): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (norm1): BatchNorm2d(129, eps=1e-05, momentum=0.1, affine=True)
  (conv2_1): Conv2d(129, 1, kernel_size=(1, 1), stride=(1, 1))
  (conv2_3): Conv2d(129, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_5): Conv2d(129, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_7): Conv2d(129, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (norm2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(4, 1, kernel_size=(1, 1), stride=(1, 1))
)

In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [13]:
from torchvision.utils import save_image
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (input, target, weight) in enumerate(train_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx == 0:
            n = 8
            comparison = torch.cat([input[:n], output[:n]])
            save_image(comparison.data.cpu(),
                     './results/train_' + str(epoch) + '.png', nrow=n)

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [14]:
from torchvision.utils import save_image
def test(epoch):
    model.eval()
    test_loss = 0
    for i, (input, target, weight) in enumerate(test_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        output = model(input)
        test_loss += loss_func(output, target, weight).data[0]
        if i == 0:
            n = 8
            comparison = torch.cat([input[:n], output[:n]])
            save_image(comparison.data.cpu(),
                     './results/test_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [15]:
torch.cuda.empty_cache()
epochs = 20
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
#


====> Epoch: 1 Average loss: 0.8473
====> Test set loss: 0.9167
====> Epoch: 2 Average loss: 0.8315
====> Test set loss: 0.9008
====> Epoch: 3 Average loss: 0.8160
====> Test set loss: 0.9246
====> Epoch: 4 Average loss: 0.8006
====> Test set loss: 0.8812
====> Epoch: 5 Average loss: 0.7787
====> Test set loss: 0.8933
====> Epoch: 6 Average loss: 0.7683
====> Test set loss: 0.7987
====> Epoch: 7 Average loss: 0.7583
====> Test set loss: 0.7996
====> Epoch: 8 Average loss: 0.7461
====> Test set loss: 0.8211
====> Epoch: 9 Average loss: 0.7454
====> Test set loss: 0.8113
====> Epoch: 10 Average loss: 0.7380
====> Test set loss: 0.7650
====> Epoch: 11 Average loss: 0.7322
====> Test set loss: 0.9139
====> Epoch: 12 Average loss: 0.7359
====> Test set loss: 0.7631
====> Epoch: 13 Average loss: 0.7310
====> Test set loss: 0.9408
====> Epoch: 14 Average loss: 0.7258
====> Test set loss: 0.7396
====> Epoch: 15 Average loss: 0.7219
====> Test set loss: 0.8886
====> Epoch: 16 Average loss: 0.72

In [16]:
torch.save(model.state_dict(), './models/segnet_1')